<a href="https://colab.research.google.com/github/panpepson/Znaki-drogowe/blob/master/znaki_d5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# D5 
# https://www.youtube.com/watch?v=w8RkFik1hUU

!pip install hyperopt

In [12]:

import pandas as pd
import numpy as np
import os 
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical


%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/DW-Matrix/Znaki-drogowe'

/content/drive/My Drive/Colab Notebooks/DW-Matrix/Znaki-drogowe


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
#robimy macierz 
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train, 
        y_train,
        batch_size = params_fit.get('batch_size', 128), # podzial wierszy na paczki
        epochs = params_fit.get('epochs',5), #liczba iteracji 
        verbose = params_fit.get('verbose',1),
        validation_data = params_fit.get('validation_data',(X_train, y_train)), #potzrebne do krzywych aby zobaczyc czy sie pzreuczyl
        callbacks = [tensorboard_callback] #do wizualizcji
     ) 
    
    #return logdir 
    return model 


def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1) #axis oznacza znajdz najwiesze prawdopodobienstwo dla jednego znku w srod maciezy
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return accuracy_score(y_test_norm, y_pred)    

In [0]:
def get_cnn_v5(input_shape, num_classes):
   return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),
                    
          Flatten(),
          Dense(1024, activation='relu'),
          Dropout(0.3), #zeby sie ie przeuczyl 
          
          Dense(1024, activation='relu'),
          Dropout(0.3), 

          Dense(num_classes, activation='softmax'),

          ])
   
#get_cnn_v4(input_shape, num_classes).summary()
#train_and_predict(get_cnn_v5(input_shape, num_classes))

In [9]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.7439 - accuracy: 0.2603 - val_loss: 1.3359 - val_accuracy: 0.5642
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 1.1187 - accuracy: 0.6388 - val_loss: 0.3142 - val_accuracy: 0.9062
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.4890 - accuracy: 0.8452 - val_loss: 0.1215 - val_accuracy: 0.9613
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2870 - accuracy: 0.9108 - val_loss: 0.0673 - val_accuracy: 0.9800
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2182 - accuracy: 0.9351 - val_loss: 0.0477 - val_accuracy: 0.9853


0.9698412698412698

In [0]:
def get_model(params):
   return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
          MaxPool2D(),
          Dropout(params['dropout_cnn_block_one']),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(params['dropout_cnn_block_two']),

          Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(params['dropout_cnn_block_three']),
                    
          Flatten(),
          Dense(1024, activation='relu'),
          Dropout(params['dropout_dense_block_one']), #zeby sie ie przeuczyl 
          
          Dense(1024, activation='relu'),
          Dropout(params['dropout_dense_block_two']), 

          Dense(num_classes, activation='softmax'),

          ])

In [0]:
def func_obj(params):
   model = get_model(params) 
   model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

   model.fit(
        X_train, 
        y_train,
        batch_size = int(params.get('batch_size', 128)), # podzial wierszy na paczki
        epochs = 5, #liczba iteracji 
        verbose = 0,
        ) 
   scope = model.evaluate(X_test, y_test)
   accuracy = scope[1] 
   print(params, 'accurancy={}'.format(accuracy))
   
   return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}  #80 bedzie -0.8 a 100 -1.0 1 jest mniejsza 

In [44]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10), # od 50 do 200 krok 20 
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest, #algorytm
    3,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.1882 - accuracy: 0.9375

 15/138 [==>...........................]
 - ETA: 0s - loss: 0.1294 - accuracy: 0.9646

 28/138 [=====>........................]
 - ETA: 0s - loss: 0.1517 - accuracy: 0.9688

 41/138 [=======>......................]
 - ETA: 0s - loss: 0.1086 - accuracy: 0.9787

 53/138 [==========>...................]
 - ETA: 0s - loss: 0.1445 - accuracy: 0.9735

 65/138 [=============>................]
 - ETA: 0s - loss: 0.1755 - accuracy: 0.9611
